**L'objectif ici est de regrouper toutes les données dans un maxi fichier qui comprend toutes les réponses (de base) aux questions, mais également les réponses traitées (par spacy)**

In [0]:
import pandas as pd

# Récupération des réponses des quatres fichiers grand débat
# A chaque fois on supprime toutes les réponses aux questions qui ne sont pas libres
# Pour chaque réponse on note dans quel fichier elle était à l'origine
demo_data = pd.read_csv("grand_debat/grand_debat_democratie.csv", sep='#')
demo_data = demo_data[demo_data.libre==True]
demo_data['parent']="democratie"

ecolo_data = pd.read_csv("grand_debat/grand_debat_ecologie.csv", sep='#')
ecolo_data = ecolo_data[ecolo_data.libre==True]
ecolo_data['parent']="ecologie"

fisc_data = pd.read_csv("grand_debat/grand_debat_fiscalite.csv", sep='#')
fisc_data = fisc_data[fisc_data.libre==True]
fisc_data['parent']="fiscalite"

serv_data = pd.read_csv("grand_debat/grand_debat_services.csv", sep='#')
serv_data = serv_data[serv_data.libre==True]
serv_data['parent']="services"

# On regroupe toutes les réponses dans un même fichier
documents = demo_data.append(ecolo_data).append(fisc_data).append(serv_data)

FileNotFoundError: [Errno 2] File b'grand_debat/grand_debat_democratie.csv' does not exist: b'grand_debat/grand_debat_democratie.csv'

In [0]:
import spacy # on utilise spacy pour traiter les documents

nlp = spacy.load('fr_core_news_sm', disable=['ner', 'parser']) 

def clean_component(doc):
    """ nettoie le text. Lemmatize, lowercase, enlève la punctuation et les stopwords (mots trop fréquents) """
    doc = [d.lemma_.lower() for d in doc if (not d.is_stop and d.is_alpha)]
    return doc

def pipe_clean(docs, **kwargs):
    for doc in docs:
        yield clean_component(doc)

clean_component.pipe = pipe_clean

nlp.add_pipe(clean_component, name='cleaner', after='tagger') # on ajoute au processus de traitement du texte la fonction pipe_clean créé au dessus

ModuleNotFoundError: No module named 'spacy'

In [0]:
# On ajoute a documents une liste 'tokens' qui est, pour chaques reponse libre, une liste des termes lemmatizés de la réponse
documents["tokens"] = list(nlp.pipe(documents["reponse"].astype(str).to_list()))

In [0]:
# On sauvegarde dans un fichier pickle l'ensemble des données
documents.to_pickle('grand_debat_data.pk1')